In [ ]:
import os 
import pandas as pd
import numpy as np

import IPython.display as ipd
import warnings
import librosa
import librosa.display
import librosa.feature
import matplotlib.pyplot as plt
from tqdm import tqdm
import utils

import processing
import event_manager
import model_training
import epoching

import pickle
import model_prepare
warnings.filterwarnings('ignore')

In [ ]:
audio_folder = '../../data/ICBHI_final_database'
list_audio_files = processing.get_list_recording(audio_folder)
print('Found {} recording files'.format(len(list_audio_files)))

In [ ]:
fs = 22050
n_fft =2048
hop_length=512
win_len=1
win_shift=0.250
df_label_list=list()
epochs_list = []
for audio_file in tqdm(list_audio_files):
    try:
        data_file = processing.load_file_from_recording_name(audio_folder,audio_file)
        data_filtered,df_label = processing.preprocess_data(data_file,fs=fs, annotations='event')
        df_label = event_manager.fill_gap_event_frame(df_label,data_filtered)
        df_label = event_manager.segment_event_annotation(data_filtered,win_len=win_len,win_shift=win_shift,threshold_class=[1,None,None])
        df_label['file']=audio_file
        df_label = model_prepare.select_nsample_normal(df_label)
        epochs_ = epoching.get_epoching_from_label_data(data_filtered,df_label,constant_values=0)
        X,f = processing.apply_mel_on_epochs(epochs_,n_fft=n_fft,
                                             hop_length=hop_length,
                                             fs=data_filtered['fs'])     
        epochs_list.append(X)
        df_label_list.append(df_label)
    except:
        print(audio_file)


In [ ]:
X,df_all_label = model_prepare.prepare_data_time_for_modeling(epochs_list,df_label_list)

In [ ]:
import pickle 
with open('../../data/data_model/data_nothreshold_win1s_sftf.pkl','wb') as f:
    pickle.dump(
        {
        'feature': X,
        'label':df_all_label,
        },f)